# PowerBI Workspace Migration to Fabric Capacity

This notebook provides a step-by-step guide to migrating Power BI workspaces to F Capacity based on Capacity and Workspace IDs.

## Prerequisites

Ensure you have the following prerequisites before running the notebook:

1. **Python Packages**: 
   - Install the required Python packages:
     ```bash
     pip install msal
     pip install azure-identity azure-keyvault-secrets
     ```

2. **Azure Credentials**:
   - Set the following environment variables with your Azure credentials:
     - `AZURE_CLIENT_ID`
     - `AZURE_TENANT_ID`
     - `AZURE_CLIENT_SECRET`

3. **Azure Key Vault**:
   - Your Azure Key Vault should store the client secret needed for authentication.
   - Set the Key Vault name and secret name in the notebook.

## Option 1: Migrates Workspaces Based on Capacity and Workspace IDs

### Step 1: Set Environment Variables
Set the environment variables for `AZURE_CLIENT_ID`, `AZURE_TENANT_ID`, and `AZURE_CLIENT_SECRET`.

In [2]:
import os
# Set the following environment variables with the values of your Azure AD application for testing purposes
os.environ['AZURE_CLIENT_ID'] = '<CLIENT ID>' # Application (client) ID of your Azure AD application
os.environ['AZURE_TENANT_ID'] = '<TENANT ID>' # Directory (tenant) ID of your Azure AD tenant
os.environ['AZURE_CLIENT_SECRET'] = '<CLIENT SECRET>' # Client secret of your Azure AD application




### Step 2: Retrieve Client Secret from Azure Key Vault
Fetch the client secret from Azure Key Vault using the `azure-identity` and `azure-keyvault-secrets` packages.

In [3]:
from azure.identity import EnvironmentCredential
from azure.keyvault.secrets import SecretClient

key_vault_name = "<KEY_VAULT_NAME>" # Replace with your Key Vault name
secret_name = "<KEY_VAULT_SECRET>" # Replace with your secret name

vault_url = f"https://{key_vault_name}.vault.azure.net"
credential = EnvironmentCredential()
secret_client = SecretClient(vault_url=vault_url, credential=credential)

retrieved_secret = secret_client.get_secret(secret_name)
client_secret = retrieved_secret.value

### Step 3: Acquire Token for Microsoft Fabric API
Use the `msal` package to acquire an access token for the Microsoft Fabric API.

In [4]:
from msal import ConfidentialClientApplication

tenant_id = os.getenv('AZURE_TENANT_ID')
client_id = os.getenv('AZURE_CLIENT_ID')
authority = f"https://login.microsoftonline.com/{tenant_id}"
scopes = ["https://api.fabric.microsoft.com/.default"]

app = ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)
result = app.acquire_token_for_client(scopes)

if 'access_token' in result:
    access_token = result['access_token']
else:
    raise Exception("Token acquisition failed:", result.get("error"), result.get("error_description"))

### Step 4: Retrieve Capacities and Workspaces
Get the list of capacities and workspaces from the Microsoft Fabric API.

In [5]:
import requests

headers = {'Authorization': 'Bearer ' + access_token}

capacities_url = "https://api.fabric.microsoft.com/v1/capacities"
capacities_response = requests.get(capacities_url, headers=headers)
capacities = capacities_response.json()["value"] if capacities_response.status_code == 200 else []

workspaces_url = "https://api.fabric.microsoft.com/v1/workspaces"
workspaces_response = requests.get(workspaces_url, headers=headers)
workspaces = workspaces_response.json()["value"] if workspaces_response.status_code == 200 else []

#### Display Capacities
Print the details of each capacity.

In [6]:
if capacities:
    print("Capacities:")
    for capacity in capacities:
        print(f"ID: {capacity['id']}")
        print(f"Name: {capacity['displayName']}")
        print(f"SKU: {capacity['sku']}")
        print(f"Region: {capacity['region']}")
        print(f"State: {capacity['state']}")
        print("--------")
else:
    print("Failed to get capacities:", capacities_response.status_code, capacities_response.text)

Capacities:
ID: e1be100d-d431-47cf-8de9-sadsadsadaaa
Name: fabriccapacitydkndev
SKU: F2
Region: West US 3
State: Active
--------
ID: f48a3ab6-c5ee-40af-be8d-dsadasfdsafsa
Name: Premium Per User - Reserved
SKU: PP3
Region: West US 3
State: Active
--------


#### Display Workspaces
Print the details of each workspace.

In [7]:
if workspaces:
    print("Workspaces:")
    workspace_list = []
    for workspace in workspaces:
        workspace_list.append({
            'id': workspace['id'],
            'name': workspace['displayName'],
            'description': workspace['description'],
            'type': workspace['type'],
            'capacityId': workspace.get('capacityId', 'N/A')
        })
        print(f"ID: {workspace['id']}")
        print(f"Name: {workspace['displayName']}")
        print(f"Description: {workspace['description']}")
        print(f"Type: {workspace['type']}")
        print(f"Capacity ID: {workspace.get('capacityId', 'N/A')}")
        print("--------")
else:
    print("Failed to get workspaces:", workspaces_response.status_code, workspaces_response.text)

Workspaces:
ID: 5dfba9f1-3c4a-48e8-87ed-1111133333333
Name: wksp-dkn-dev
Description: the dev workspace
Type: Workspace
Capacity ID: e1be100d-d431-47cf-8de9-sadsadsadaaa
--------
ID: 10271529-d210-4423-b1d1-22222222222
Name: wksp-dkn-vnet-dev
Description: 
Type: Workspace
Capacity ID: 2e931b9d-dbbf-41b9-a495-3aaaaaaaa222
--------
ID: 6ef1566f-4c95-4d1e-b2ed-44444444444
Name: wksp-dkn-test-1
Description: 
Type: Workspace
Capacity ID: 2e931b9d-dbbf-41b9-a495-3aaaaaaaa222
--------


### Step 5: Select and Assign Workspaces to Capacities
Select the capacity and workspaces for assignment and assign them accordingly.

In [9]:
if capacities:
    selected_capacity_id = input("Enter Capacity ID to assign workspaces to: ")

    print("Select workspaces to assign (comma-separated IDs):")
    for workspace in workspace_list:
        print(f"ID: {workspace['id']}, Name: {workspace['name']}, Description: {workspace['description']}, Capacity ID: {workspace['capacityId']}")
    selected_workspace_ids = input("Enter Workspace IDs: ").split(",")

    # Assign workspaces to the selected capacity
    for workspace_id in selected_workspace_ids:
        assign_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/assignToCapacity"
        data = {"capacityId": selected_capacity_id}
        assign_response = requests.post(assign_url, headers=headers, json=data)

        if assign_response.status_code in [200, 202]:
            print(f"Workspace {workspace_id} assigned successfully to capacity {selected_capacity_id}.")
        else:
            print(f"Failed to assign workspace {workspace_id}: {assign_response.status_code} {assign_response.text}")
else:
    print("No capacities available to select.")

Select workspaces to assign (comma-separated IDs):
ID: 5dfba9f1-3c4a-48e8-87ed-1111133333333, Name: wksp-dkn-dev, Description: the dev workspace, Capacity ID: e1be100d-d431-47cf-8de9-sadsadsadaaa
ID: 10271529-d210-4423-b1d1-22222222222, Name: wksp-dkn-vnet-dev, Description: , Capacity ID: 2e931b9d-dbbf-41b9-a495-3aaaaaaaa222
ID: 6ef1566f-4c95-4d1e-b2ed-44444444444, Name: wksp-dkn-test-1, Description: , Capacity ID: 2e931b9d-dbbf-41b9-a495-3aaaaaaaa222
Workspace 6ef1566f-4c95-4d1e-b2ed-44444444444 assigned successfully to capacity e1be100d-d431-47cf-8de9-sadsadsadaaa.


## Option 2: Migrates Workspaces Based on Capacity ID Mapping

### Step 1: Set Environment Variables
Set the environment variables for `AZURE_CLIENT_ID`, `AZURE_TENANT_ID`, and `AZURE_CLIENT_SECRET`.

In [10]:
import os

# Set the following environment variables with the values of your Azure AD application for testing purposes
os.environ['AZURE_CLIENT_ID'] = '<CLIENT ID>' # Application (client) ID of your Azure AD application
os.environ['AZURE_TENANT_ID'] = '<TENANT ID>' # Directory (tenant) ID of your Azure AD tenant
os.environ['AZURE_CLIENT_SECRET'] = '<CLIENT SECRET>' # Client secret of your Azure AD application


### Step 2: Retrieve Client Secret from Azure Key Vault
Fetch the client secret from Azure Key Vault using the `azure-identity` and `azure-keyvault-secrets` packages.

In [11]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

key_vault_name = "<KEY_VAULT_NAME>" # Replace with your Key Vault name
secret_name = "<KEY_VAULT_SECRET>" # Replace with your secret name

vault_url = f"https://{key_vault_name}.vault.azure.net"
credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url=vault_url, credential=credential)

retrieved_secret = secret_client.get_secret(secret_name)
client_secret = retrieved_secret.value

### Step 3: Acquire Token for Microsoft Fabric API
Use the `msal` package to acquire an access token for the Microsoft Fabric API.

In [12]:
from msal import ConfidentialClientApplication

tenant_id = os.getenv('AZURE_TENANT_ID')
client_id = os.getenv('AZURE_CLIENT_ID')
authority = f"https://login.microsoftonline.com/{tenant_id}"
scopes = ["https://api.fabric.microsoft.com/.default"]

app = ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)
result = app.acquire_token_for_client(scopes)

if 'access_token' in result:
    access_token = result['access_token']
else:
    raise Exception("Token acquisition failed:", result.get("error"), result.get("error_description"))

### Step 4: Retrieve Capacities and Workspaces
Get the list of capacities and workspaces from the Microsoft Fabric API.

In [13]:
import requests

headers = {'Authorization': 'Bearer ' + access_token}

capacities_url = "https://api.fabric.microsoft.com/v1/capacities"
capacities_response = requests.get(capacities_url, headers=headers)
capacities = capacities_response.json()["value"] if capacities_response.status_code == 200 else []

workspaces_url = "https://api.fabric.microsoft.com/v1/workspaces"
workspaces_response = requests.get(workspaces_url, headers=headers)
workspaces = workspaces_response.json()["value"] if workspaces_response.status_code == 200 else []

#### Display Capacities
Print the details of each capacity.

In [14]:
if capacities:
    print("Capacities:")
    for capacity in capacities:
        print(f"ID: {capacity['id']}")
        print(f"Name: {capacity['displayName']}")
        print(f"SKU: {capacity['sku']}")
        print(f"Region: {capacity['region']}")
        print(f"State: {capacity['state']}")
        print("--------")
else:
    print("Failed to get capacities:", capacities_response.status_code, capacities_response.text)

Capacities:
ID: e1be100d-d431-47cf-8de9-sadsadsadaaa
Name: fabriccapacitydkndev
SKU: F2
Region: West US 3
State: Active
--------
ID: f48a3ab6-c5ee-40af-be8d-dsadasfdsafsa
Name: Premium Per User - Reserved
SKU: PP3
Region: West US 3
State: Active
--------


#### Display Workspaces
Print the details of each workspace.

In [15]:
if workspaces:
    print("Workspaces:")
    for workspace in workspaces:
        print(f"ID: {workspace['id']}")
        print(f"Name: {workspace['displayName']}")
        print(f"Description: {workspace['description']}")
        print(f"Type: {workspace['type']}")
        print(f"Capacity ID: {workspace.get('capacityId', 'N/A')}")
        print("--------")
else:
    print("Failed to get workspaces:", workspaces_response.status_code, workspaces_response.text)

Workspaces:
ID: 5dfba9f1-3c4a-48e8-87ed-1111133333333
Name: wksp-dkn-dev
Description: the dev workspace
Type: Workspace
Capacity ID: e1be100d-d431-47cf-8de9-sadsadsadaaa
--------
ID: 10271529-d210-4423-b1d1-22222222222
Name: wksp-dkn-vnet-dev
Description: 
Type: Workspace
Capacity ID: 2e931b9d-dbbf-41b9-a495-3aaaaaaaa222
--------
ID: 6ef1566f-4c95-4d1e-b2ed-44444444444
Name: wksp-dkn-test-1
Description: 
Type: Workspace
Capacity ID: e1be100d-d431-47cf-8de9-sadsadsadaaa
--------


### Step 5: Map Workspaces to New Capacities
Define the mapping from legacy capacities to new Fabric capacities using capacity IDs and display the mappings.

In [16]:
capacity_mapping = {
  #  "2e931b9d-dbbf-41b9-5555-3333333333": "e1be100d-d431-47cf-8889-4444444444",
    #"e1be100d-d431-47cf-8de9-sadsadsadaaa": "2e931b9d-dbbf-41b9-a495-3aaaaaaaa222" #f2 sku -> trial sku,
    "2e931b9d-dbbf-41b9-a495-3aaaaaaaa222": "e1be100d-d431-47cf-8de9-sadsadsadaaa" #trial sku -> f2 sku
    # Add more mappings as needed
}

mappings = []
for workspace in workspaces:
    workspace_name = workspace.get("displayName")
    legacy_capacity_id = workspace.get("capacityId")
    new_capacity_id = capacity_mapping.get(legacy_capacity_id)

    if new_capacity_id:
        mappings.append((workspace_name, workspace['id'], legacy_capacity_id, new_capacity_id))

# Display the mappings
if mappings:
    print("Workspaces and their mapped capacities:")
    for mapping in mappings:
        print(f"Workspace: {mapping[0]} (ID: {mapping[1]})")
        print(f"  Legacy Capacity ID: {mapping[2]}")
        print(f"  New Capacity ID: {mapping[3]}")
        print("  --------")
else:
    print("No valid mappings found.")

Workspaces and their mapped capacities:
Workspace: wksp-dkn-vnet-dev (ID: 10271529-d210-4423-b1d1-22222222222)
  Legacy Capacity ID: 2e931b9d-dbbf-41b9-a495-3aaaaaaaa222
  New Capacity ID: e1be100d-d431-47cf-8de9-sadsadsadaaa
  --------


### Step 6: Migrate Workspaces
Confirm the mappings and proceed with the migration of workspaces to the new capacities.

In [18]:
confirm = input("Do you want to proceed with the migration? (yes/no): ").strip().lower()
if confirm != 'yes':
    print("Migration aborted by user.")
    exit()

for mapping in mappings:
    workspace_name, workspace_id, legacy_capacity_id, new_capacity_id = mapping
    assign_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/assignToCapacity"
    data = {"capacityId": new_capacity_id}
    assign_response = requests.post(assign_url, headers=headers, json=data)

    if assign_response.status_code in [200, 202]:
        print(f"Workspace {workspace_name} (ID: {workspace_id}) assigned successfully to new capacity (ID: {new_capacity_id}).")
    else:
        print(f"Failed to assign workspace {workspace_name} (ID: {workspace_id}): {assign_response.status_code} {assign_response.text}")

    # Optional: Delay between migrations to manage load
    #time.sleep(1)

Workspace wksp-dkn-vnet-dev (ID: 10271529-d210-4423-b1d1-22222222222) assigned successfully to new capacity (ID: e1be100d-d431-47cf-8de9-sadsadsadaaa).


## Notes
- Ensure that the capacity mappings are correctly defined.
- The script includes debugging prints to help verify each step.
- The `time.sleep(1)` function call can be adjusted or removed based on the load and performance requirements.

## License
This project is licensed under the MIT License.